In [1]:
# Installation of dependencies
!pip install llama-index-multi-modal-llms-gemini
!pip install llama-index-vector-stores-qdrant
!pip install llama-index-embeddings-gemini
!pip install llama-index-llms-gemini
!pip install llama-index 'google-generativeai>=0.3.0' matplotlib qdrant_client
!pip install nest_asyncio

  Using cached llama_index_vector_stores_qdrant-0.5.0-py3-none-any.whl.metadata (767 bytes)
  Using cached qdrant_client-1.13.3-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.5/264.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 21.3 MB/s eta 0:00:00


In [2]:
# Import necessary dependencies
import os
from google.colab import userdata
from llama_index.llms.gemini import Gemini
from llama_index.core.llms import ChatMessage, ImageBlock
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import time
import asyncio
import nest_asyncio  # Fix for Jupyter event loop issue
from pathlib import Path
from pydantic import BaseModel
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.core.program import MultiModalLLMCompletionProgram
from llama_index.core.output_parsers import PydanticOutputParser
from llama_index.core import SimpleDirectoryReader
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import Settings
from llama_index.core import StorageContext
import qdrant_client
import asyncio
import aiohttp  # For handling image URLs asynchronously

# Apply the nest_asyncio patch
nest_asyncio.apply()

In [3]:
# Configure API key and initialize model
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('api_key')

os.environ['GOOGLE_API_KEY']=userdata.get('api_key')
model = Gemini(model_name="models/gemini-1.5-pro")

<ipython-input-3-7927d79be21b>:6: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  model = Gemini(model_name="models/gemini-1.5-pro")


In [5]:
# Core system
async def chatbot_response(user_input=None, image_url=None):
    try:
        if image_url:
            # Step 1: Create message with image URL
            msg = ChatMessage("Describe the content of this image in a detailed caption.")
            msg.blocks.append(ImageBlock(url=image_url))

            # Step 2: Generate caption for the image
            caption_response = await asyncio.to_thread(model.chat, messages=[msg])
            image_caption = caption_response.message.content

            # Step 3: Use the caption to generate a conversation response
            chat_prompt = ChatMessage(f"Continue the conversation based on this image description: {image_caption}")
            chat_response = await asyncio.to_thread(model.chat, messages=[chat_prompt])
            return chat_response.message.content

        elif user_input:
            response = await asyncio.to_thread(model.chat, messages=[ChatMessage(user_input)])
            return response.message.content

        else:
            return "Please provide an input (text or image URL)."

    except Exception as e:
        return "An error occurred. Please try again."

def run_async_task(task):
    """Runs async functions safely in both normal Python and Jupyter/Colab."""
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    return loop.run_until_complete(task)

def main():
    """
    Command-line chatbot interface.
    """
    print("Welcome to the Chatbot! Type 'exit' to quit.")

    while True:
        user_input = input("\nEnter text message (or type 'img' to provide an image URL): ").strip()

        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        elif user_input.lower() == "img":
            image_url = input("Enter the image URL: ").strip()
            response = run_async_task(chatbot_response(image_url=image_url))
            print("\nBot:", response)

        else:
            response = run_async_task(chatbot_response(user_input=user_input))
            print("\nBot:", response)

if __name__ == "__main__":
    main()

Welcome to the Chatbot! Type 'exit' to quit.

Enter text message (or type 'img' to provide an image URL): hi there!

Bot: Hi there! How can I help you today?


Enter text message (or type 'img' to provide an image URL): tell me your service

Bot: I am a large language model, trained by Google.  I can provide a variety of services, including:

* **Generating text:** I can write different creative text formats, like poems, code, scripts, musical pieces, email, letters, etc. I can also translate languages, write different kinds of creative content, and answer your questions in an informative way, even if they are open ended, challenging, or strange.
* **Summarizing text:** I can provide concise summaries of longer pieces of text.
* **Answering questions:** I can answer your questions based on the knowledge I have been trained on.  I can access and process information from the real world through Google Search and keep my response consistent with search results.
* **Translating languages:**